# Load Required Packages

In [ ]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install chart_studio
    !pip install --upgrade autopep8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 245 kB 4.6 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 102 kB 83.7 MB/s 
     |████████████████████████████████| 4.7 MB 75.5 MB/s 
     |████████████████████████████████| 690 kB 84.2 MB/s 
     |████████████████████████████████| 355 kB 83.7 MB/s 
     |████████████████████████████████| 296 kB 80.9 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=c7cde2321809eca8f2c18a0147b6e7c78ed6b71b602ac953d3b1e162d7bad439
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
Successfully built htmlmin
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas-profiling
    Fou

In [ ]:
import pandas as pd

In [ ]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim_models
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(messBage)s', level=logging.INFO)

# Read in data

In [ ]:
model_ows = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ows_tweet_topics.csv')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ows = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ows-attributes.csv', on_bad_lines='skip')
ows.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,...,Source,clean_text,text_lemmatized,lemmatized_tokens,mentioned,hashtags,is_retweet,has_multimedia,has_mention,is_modifiedTweet
0,0,121813144174727168,RT @AnonKitsu: ALERT!!!!!!!!!!COPS ARE KETTLIN...,http://a2.twimg.com/profile_images/1539375713/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,Anonops_Cop,...,&lt;a href=&quot;http://twitter.com/&quot;&gt;...,alert cops are kettling protesters in park he...,"['alert', 'cops', 'are', 'kettling', 'proteste...","['alert', 'cops', 'kettling', 'protester', 'pa...",[],"['#occupywallstreet', '#ows', '#occupyny']",True,False,False,False
1,1,121813146137657344,@jamiekilstein @allisonkilkenny Interesting in...,http://a2.twimg.com/profile_images/1574715503/...,2011-10-06,5,4,2011-10-06 05:04:51,N;,KittyHybrid,...,&lt;a href=&quot;http://twitter.com/&quot;&gt;...,interesting interview never aired wonder wh...,"['interesting', 'interview', 'never', 'aired',...","['interesting', 'interview', 'never', 'aired',...","['@jamiekilstein', '@allisonkilkenny']",['#ows'],False,True,True,False
2,2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,http://a1.twimg.com/profile_images/1241412831/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,nerdsherpa,...,&lt;a href=&quot;http://www.echofon.com/&quot;...,right those guys have victory condition re...,"['right', 'those', 'guys', 'have', 'victory', ...","['right', 'guys', 'victory', 'condition', 'reg...",['@Seductivpancake'],['#ows'],False,False,True,False
3,3,121813150701072385,RT @bembel &quot;Occupy Wall Street&quot; als ...,http://a0.twimg.com/profile_images/1106399092/...,2011-10-06,5,4,2011-10-06 05:04:52,N;,hamudistan,...,&lt;a href=&quot;http://levelupstudio.com&quot...,occupy wall street als linke antwort auf die ...,"['occupy', 'wall', 'street', 'als', 'linke', '...","['occupy', 'wall', 'street', 'als', 'linke', '...",[],"['#occupywallstreet', '#ows']",True,True,False,False
4,4,121813163778899968,#ows White shirt= Brown shirt.,http://a2.twimg.com/profile_images/1568117871/...,2011-10-06,5,4,2011-10-06 05:04:56,N;,kl_knox,...,&lt;a href=&quot;http://twitter.com/&quot;&gt;...,#ows white shirt brown shirt,"['#ows', 'white', 'shirt', 'brown', 'shirt']","['#ows', 'white', 'shirt', 'brown', 'shirt']",[],['#ows'],False,False,False,False


In [ ]:
model_ows = ows.drop(columns=['Unnamed: 0', 'Profile Image URL', 'Hour', 'Minute', 'Geo', 'From User', 'From User ID', 'Language', 'To User', 'To User ID', 'Source', 'mentioned', 'hashtags', 'has_mention', 'is_modifiedTweet'])
model_ows.head(10)

,Twitter ID,Text,Day,Created At,clean_text,text_lemmatized,lemmatized_tokens,is_retweet,has_multimedia
0,121813144174727168,RT @AnonKitsu: ALERT!!!!!!!!!!COPS ARE KETTLIN...,2011-10-06,2011-10-06 05:04:51,alert cops are kettling protesters in park he...,"['alert', 'cops', 'are', 'kettling', 'proteste...","['alert', 'cops', 'kettling', 'protester', 'pa...",True,False
1,121813146137657344,@jamiekilstein @allisonkilkenny Interesting in...,2011-10-06,2011-10-06 05:04:51,interesting interview never aired wonder wh...,"['interesting', 'interview', 'never', 'aired',...","['interesting', 'interview', 'never', 'aired',...",False,True
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,2011-10-06,2011-10-06 05:04:52,right those guys have victory condition re...,"['right', 'those', 'guys', 'have', 'victory', ...","['right', 'guys', 'victory', 'condition', 'reg...",False,False
3,121813150701072385,RT @bembel &quot;Occupy Wall Street&quot; als ...,2011-10-06,2011-10-06 05:04:52,occupy wall street als linke antwort auf die ...,"['occupy', 'wall', 'street', 'als', 'linke', '...","['occupy', 'wall', 'street', 'als', 'linke', '...",True,True
4,121813163778899968,#ows White shirt= Brown shirt.,2011-10-06,2011-10-06 05:04:56,#ows white shirt brown shirt,"['#ows', 'white', 'shirt', 'brown', 'shirt']","['#ows', 'white', 'shirt', 'brown', 'shirt']",False,False
5,121813169999065088,RT @TheNewDeal: The #NYPD are Out of Control. ...,2011-10-06,2011-10-06 05:04:57,the #nypd are out of control is this free co...,"['the', '#nypd', 'are', 'out', 'of', 'control'...","['#nypd', 'control', 'free', 'country', 'middl...",True,False
6,121813190454689792,1912 prediction re: founding of #Fed : Then we...,2011-10-06,2011-10-06 05:05:02,1912 prediction re founding of #fed then we s...,"['1912', 'prediction', 're', 'founding', 'of',...","['1912', 'prediction', 'founding', '#fed', 'sh...",False,True
7,121813195924058113,Wall Street Protests Marred by Anti-Semitism h...,2011-10-06,2011-10-06 05:05:03,wall street protests marred by anti semitism #ows,"['wall', 'street', 'protest', 'marred', 'by', ...","['wall', 'street', 'protest', 'marred', 'anti'...",False,True
8,121813197404639232,#OWS @cynthiaboaz 8 rules for Satyagrahi activ...,2011-10-06,2011-10-06 05:05:04,#ows rules for satyagrahi activism gene sharp...,"['#ows', 'rule', 'for', 'satyagrahi', 'activis...","['#ows', 'rule', 'satyagrahi', 'activism', 'ge...",False,True
9,121813198935560192,RT @ForseenUnNamed: Btw. Welcome to day 20 #o...,2011-10-06,2011-10-06 05:05:04,btw welcome to day #ows stay strong,"['btw', 'welcome', 'to', 'day', '#ows', 'stay'...","['btw', 'welcome', 'day', '#ows', 'stay', 'str...",True,False


In [ ]:
model_ows = model_ows[model_ows['is_retweet']==False]
model_ows = model_ows[model_ows['has_multimedia']==False]
model_ows

,Twitter ID,Text,Day,Created At,clean_text,text_lemmatized,lemmatized_tokens,is_retweet,has_multimedia
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,2011-10-06,2011-10-06 05:04:52,right those guys have victory condition re...,"['right', 'those', 'guys', 'have', 'victory', ...","['right', 'guys', 'victory', 'condition', 'reg...",False,False
4,121813163778899968,#ows White shirt= Brown shirt.,2011-10-06,2011-10-06 05:04:56,#ows white shirt brown shirt,"['#ows', 'white', 'shirt', 'brown', 'shirt']","['#ows', 'white', 'shirt', 'brown', 'shirt']",False,False
13,121813232523558912,@AnonyOps @danielhewitt Wow. Self-defense is ...,2011-10-06,2011-10-06 05:05:12,wow self defense is obviously needed to prot...,"['wow', 'self', 'defense', 'is', 'obviously', ...","['wow', 'self', 'defense', 'obviously', 'neede...",False,False
14,121813245488140288,@HumanityCritic i'm worried that the #ows sell...,2011-10-06,2011-10-06 05:05:15,m worried that the #ows sells out to the ham...,"['m', 'worried', 'that', 'the', '#ows', 'sell'...","['worried', '#ows', 'sell', 'hamsher', 'norqui...",False,False
23,121813284704882688,@LibertyBelle4 LOL! Its sneaking up on a lot o...,2011-10-06,2011-10-06 05:05:24,lol its sneaking up on lot of people but ...,"['lol', 'its', 'sneaking', 'up', 'on', 'lot', ...","['lol', 'sneaking', 'lot', 'people', 'sure', '...",False,False
...,...,...,...,...,...,...,...,...,...
77828288,170810569018974208,This is the sorry situation 10 years after the...,2012-02-18,2012-02-18 10:03:07,this is the sorry situation years after the cr...,"['this', 'is', 'the', 'sorry', 'situation', 'y...","['sorry', 'situation', 'year', 'creation', 'tr...",False,False
77828293,170811007516672000,Man's knowledge makes another leap through the...,2012-02-18,2012-02-18 10:04:52,man knowledge makes another leap through the t...,"['man', 'knowledge', 'make', 'another', 'leap'...","['man', 'knowledge', 'make', 'another', 'leap'...",False,False
77828294,170811073648279552,When we give any president - one man - too muc...,2012-02-18,2012-02-18 10:05:08,when we give any president one man too much po...,"['when', 'we', 'give', 'any', 'president', 'on...","['give', 'president', 'one', 'man', 'much', 'p...",False,False
77828295,170811301411553281,NYC forecast Tue 2/21/12: Partly cloudy. High ...,2012-02-18,2012-02-18 10:06:02,nyc forecast tue partly cloudy high temp 48f...,"['nyc', 'forecast', 'tue', 'partly', 'cloudy',...","['nyc', 'forecast', 'tue', 'partly', 'cloudy',...",False,False


#Base Model

## Base Model Creation

In [ ]:
def remove_top_words(word):
    word = re.sub('(,\s)?#ows(,\s)?','',word)
    word = re.sub('(,\s)?people(,\s)?','',word)
    word = re.sub('(,\s)?#occupywallstreet(,\s)?','',word)
    word = re.sub('(,\s)?#occupy(,\s)?','',word)
    word = re.sub('(,\s)?#tcot(,\s)?','',word)
    word = re.sub('(,\s)?like(,\s)?','',word)
    word = re.sub('(,\s)?get(,\s)?','',word)
    word = re.sub('(,\s)?#p2(,\s)?','',word)
    word = re.sub('\'\',','', word)
    word = re.sub('\'\s\',','', word)
    word = re.sub('\'\s', '\'', word)
    word = re.sub('\s\'', '\'', word)
    word = re.sub('\s', '', word)
    return word

In [ ]:
model_ows['lemmatized_tokens'] = model_ows['lemmatized_tokens'].astype(str)
model_ows['lemmatized_tokens'] = model_ows['lemmatized_tokens'].apply(lambda x: remove_top_words(x))
model_ows['lemmatized_tokens'] = [re.sub('["\[\]\']+','', str(x)).split(',') for x in model_ows['lemmatized_tokens']]
model_ows

,Twitter ID,Text,Day,Created At,clean_text,text_lemmatized,lemmatized_tokens,is_retweet,has_multimedia
2,121813150000619521,@Seductivpancake Right! Those guys have a vict...,2011-10-06,2011-10-06 05:04:52,right those guys have victory condition re...,"['right', 'those', 'guys', 'have', 'victory', ...","[right, guys, victory, condition, regime, chan...",False,False
4,121813163778899968,#ows White shirt= Brown shirt.,2011-10-06,2011-10-06 05:04:56,#ows white shirt brown shirt,"['#ows', 'white', 'shirt', 'brown', 'shirt']","[white, shirt, brown, shirt]",False,False
13,121813232523558912,@AnonyOps @danielhewitt Wow. Self-defense is ...,2011-10-06,2011-10-06 05:05:12,wow self defense is obviously needed to prot...,"['wow', 'self', 'defense', 'is', 'obviously', ...","[wow, self, defense, obviously, needed, protec...",False,False
14,121813245488140288,@HumanityCritic i'm worried that the #ows sell...,2011-10-06,2011-10-06 05:05:15,m worried that the #ows sells out to the ham...,"['m', 'worried', 'that', 'the', '#ows', 'sell'...","[worried, sell, hamsher, norquist, spitefuck, ...",False,False
23,121813284704882688,@LibertyBelle4 LOL! Its sneaking up on a lot o...,2011-10-06,2011-10-06 05:05:24,lol its sneaking up on lot of people but ...,"['lol', 'its', 'sneaking', 'up', 'on', 'lot', ...","[lol, sneaking, lot, sure, undivided, atention...",False,False
...,...,...,...,...,...,...,...,...,...
77828288,170810569018974208,This is the sorry situation 10 years after the...,2012-02-18,2012-02-18 10:03:07,this is the sorry situation years after the cr...,"['this', 'is', 'the', 'sorry', 'situation', 'y...","[sorry, situation, year, creation, transportat...",False,False
77828293,170811007516672000,Man's knowledge makes another leap through the...,2012-02-18,2012-02-18 10:04:52,man knowledge makes another leap through the t...,"['man', 'knowledge', 'make', 'another', 'leap'...","[man, knowledge, make, another, leap, test, pr...",False,False
77828294,170811073648279552,When we give any president - one man - too muc...,2012-02-18,2012-02-18 10:05:08,when we give any president one man too much po...,"['when', 'we', 'give', 'any', 'president', 'on...","[give, president, one, man, much, power, build...",False,False
77828295,170811301411553281,NYC forecast Tue 2/21/12: Partly cloudy. High ...,2012-02-18,2012-02-18 10:06:02,nyc forecast tue partly cloudy high temp 48f...,"['nyc', 'forecast', 'tue', 'partly', 'cloudy',...","[nyc, forecast, tue, partly, cloudy, high, tem...",False,False


In [ ]:
# Create a id2word dictionary
id2word = Dictionary(model_ows['lemmatized_tokens'])
print(len(id2word))

270264


In [ ]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
id2word.filter_extremes(no_below=2, no_above=.4)
print(len(id2word))

99999


In [ ]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in model_ows['lemmatized_tokens']]

In [ ]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
 #ows  street  get #ows  people  like  occupy  wall  time  movement

------ Topic 1 ------
 #ows  #oo  #occupy  right  #occupyoakland  police  #occupyla  #occupywallstreet  rt  #solidarity

------ Topic 2 ------
 #ows  #p2  #tcot  #teaparty  #ronpaul  #gop  #tlot  temp  obama  nyc

------ Topic 3 ------
 #ows #ows  #ronpaul  people  money  1%  tax  government  want  #teaparty

------ Topic 4 ------
 #ows  park  police #ows  #n17  protester  city  news  march  #occupywallstreet



## Base Model Evaluation

In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

#output
# Perplexity:  -8.273208988449511

In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Coherence Score:  0.35757353209081877


## Visualization

In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(base_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.058381 -0.092615       1        1  27.688335
3      0.101043 -0.167433       2        1  24.714010
4     -0.190878  0.111575       3        1  17.314837
1     -0.108639  0.014577       4        1  17.198035
2      0.256855  0.133896       5        1  13.084783, topic_info=             Term          Freq         Total Category  logprob  loglift
152           #p2  67156.000000  67156.000000  Default  30.0000  30.0000
153         #tcot  56868.000000  56868.000000  Default  29.0000  29.0000
72482         #oo  45279.000000  45279.000000  Default  28.0000  28.0000
65      #teaparty  49906.000000  49906.000000  Default  27.0000  27.0000
2639     #ronpaul  57504.000000  57504.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
53        #occupy   8236.231247  76601.871303   Topic5  -5.3697  -0.1964
396          vote   5052.283459  11957.743217   Topic5  -5.8584   1.1722
659           gop   4426.167210   7063.980738   Topic5  -5.9908   1.5662
1630      liberty   4708.698390  13302.726022   Topic5  -5.9289   0.9952
36         people   5648.207247  84991.265391   Topic5  -5.7469  -0.6775

[429 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
84019      1  0.000739      #acta
84019      4  0.999369      #acta
1638       1  0.000440   #america
1638       2  0.001760   #america
1638       4  0.001760   #america
...      ...       ...        ...
3626       2  0.003950  something
70742      1  0.999409      stair
1660       4  0.998634       stay
1660       5  0.000762       stay
21728      3  0.998657   zuccotti

[1108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 2, 3])

# Parameter Testing

## Num_topics 

In [ ]:
coherence = []
for k in range(5,13):
    print('Round: '+str(k))
    ldamodel = LdaMulticore(corpus=corpus, num_topics=k, id2word=id2word, passes=5, random_state = 100)

    cm = CoherenceModel(model=ldamodel, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')   
                                                
    coherence.append((k,cm.get_coherence()))

Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12


In [ ]:
coherence

[(5, 0.21830385066921645),
 (6, 0.25060829677182245),
 (7, 0.3066209013915885),
 (8, 0.3062490290821571),
 (9, 0.33595443094285476),
 (10, 0.32937810114679006),
 (11, 0.33166904888109305),
 (12, 0.3816055121413326)]

# Model Iteration

## Iteration 1 (Num Topics = 11)

In [ ]:
model_iter1 = LdaMulticore(corpus=corpus, num_topics=11, id2word=id2word, passes=5, workers = 12, random_state = 100, eval_every = 1)


In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
iter1_perplexity = model_iter1.log_perplexity(corpus)
print('\nPerplexity: ', iter1_perplexity) 


Perplexity:  -8.374639172173437


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter1, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter1 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter1)


Coherence Score:  0.3976274524298429


In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter1, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.003562 -0.069066       1        1  14.758400
7     -0.095206 -0.004917       2        1  14.106389
9     -0.166065  0.063025       3        1  12.060638
3      0.084699 -0.055493       4        1  10.627289
10    -0.190235  0.026150       5        1   8.514302
6      0.007100 -0.042991       6        1   8.261678
8      0.127538 -0.002685       7        1   8.106532
1      0.169174  0.044759       8        1   7.486346
5     -0.141412  0.189459       9        1   5.621522
0     -0.015996 -0.302111      10        1   5.248936
2      0.223967  0.153870      11        1   5.207968, topic_info=                      Term          Freq         Total Category  logprob  \
13                    #ows  95963.000000  95963.000000  Default  30.0000   
152                    #p2  68068.000000  68068.000000  Default  29.0000   
153                  #tcot  57690.000000  57690.000000  Default  28.0000   
72482                  #oo  45678.000000  45678.000000  Default  27.0000   
97                  street  34275.000000  34275.000000  Default  26.0000   
...                    ...           ...           ...      ...      ...   
490                    use   2941.942239  14399.921718  Topic11  -5.4780   
261             #anonymous   2827.656323  27895.359183  Topic11  -5.5176   
318                   back   2753.017256  24372.858571  Topic11  -5.5443   
221             revolution   2256.359380   9352.977157  Topic11  -5.7433   
176      #occupywallstreet   2503.192163  54982.579906  Topic11  -5.6395   

       loglift  
13     30.0000  
152    29.0000  
153    28.0000  
72482  27.0000  
97     26.0000  
...        ...  
490     1.3668  
261     0.6660  
318     0.7742  
221     1.5330  
176    -0.1345  

[856 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
4235       3  0.004336      #15m
4235       7  0.000723      #15m
4235       8  0.994473      #15m
84019      3  0.999079     #acta
14311      4  0.998123     #alec
...      ...       ...       ...
4346       2  0.931396     world
4346       3  0.063317     world
4346       6  0.001900     world
4346       7  0.001900     world
21728      3  0.999089  zuccotti

[3320 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 10, 4, 11, 7, 9, 2, 6, 1, 3])

## Iteration 2 (Num Topics = 9)

In [ ]:
model_iter1 = LdaMulticore(corpus=corpus, num_topics=9, id2word=id2word, passes=5, workers = 12, random_state = 100, eval_every = 1)


In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
iter1_perplexity = model_iter1.log_perplexity(corpus)
print('\nPerplexity: ', iter1_perplexity) 


Perplexity:  -8.353910375230358


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter1, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter1 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter1)


Coherence Score:  0.3633141354168705


In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter1, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.108795 -0.097207       1        1  19.440723
6     -0.061075  0.050388       2        1  15.094758
4     -0.051410  0.100060       3        1  14.965059
3     -0.043683  0.130663       4        1  10.778760
5     -0.056819 -0.322669       5        1  10.627666
8      0.028250  0.028440       6        1   9.185644
1      0.141477  0.019734       7        1   7.365250
2      0.327495 -0.005334       8        1   6.490048
0     -0.175439  0.095924       9        1   6.052092, topic_info=             Term          Freq         Total Category  logprob  loglift
13           #ows  98441.000000  98441.000000  Default  30.0000  30.0000
152           #p2  66096.000000  66096.000000  Default  29.0000  29.0000
72482         #oo  44070.000000  44070.000000  Default  28.0000  28.0000
153         #tcot  55466.000000  55466.000000  Default  27.0000  27.0000
2639     #ronpaul  50316.000000  50316.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
332           day   3911.002863  22692.502399   Topic9  -5.3434   1.0465
103          like   4916.473778  54319.171803   Topic9  -5.1146   0.4025
175          want   2752.332749  33778.036811   Topic9  -5.6948   0.2974
349          time   2580.200851  35315.755489   Topic9  -5.7594   0.1883
36         people   2477.408128  85650.532304   Topic9  -5.8000  -0.7383

[722 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
1633       1  0.159020      #99%
1633       2  0.149598      #99%
1633       3  0.001609      #99%
1633       4  0.000460      #99%
1633       5  0.008273      #99%
...      ...       ...       ...
2084       8  0.829933    thanks
1102       2  0.945845      wall
1102       5  0.033245      wall
1102       6  0.020208      wall
21728      4  0.997809  zuccotti

[2498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 5, 4, 6, 9, 2, 3, 1])

## Iteration 3 (Removed Top words)

In [ ]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

In [ ]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 


Perplexity:  -8.590307592300878


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Coherence Score:  0.28411441379347646


In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(base_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.085260  0.126767       1        1  27.007307
3     -0.139874 -0.033668       2        1  22.350187
2      0.071896  0.188614       3        1  21.940173
0     -0.184135 -0.167938       4        1  18.472823
1      0.337373 -0.113776       5        1  10.229511, topic_info=             Term          Freq         Total Category  logprob  loglift
172        police  53264.000000  53264.000000  Default  30.0000  30.0000
66906         #oo  49363.000000  49363.000000  Default  29.0000  29.0000
62      #teaparty  52090.000000  52090.000000  Default  28.0000  28.0000
2381     #ronpaul  57500.000000  57500.000000  Default  27.0000  27.0000
94         street  34412.000000  34412.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
604            la   5964.649729  24597.005405   Topic5  -5.3124   0.8631
132         obama   4653.297900  26946.637936   Topic5  -5.5607   0.5236
121    #anonymous   4557.145822  27571.599626   Topic5  -5.5816   0.4798
2916    president   3316.684317   7463.422910   Topic5  -5.8993   1.4688
194    government   3405.348207  22402.524853   Topic5  -5.8729   0.3961

[419 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
121       1  0.018280  #anonymous
121       2  0.198646  #anonymous
121       3  0.615488  #anonymous
121       4  0.002285  #anonymous
121       5  0.165279  #anonymous
...     ...       ...         ...
665       4  0.976643    zuccotti
665       5  0.000333    zuccotti
3314      2  0.002385     zucotti
3314      4  0.997404     zucotti
3314      5  0.000341     zucotti

[1153 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])

In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
 #teaparty #ronpaul want think us #gop #ndaa party obama

------ Topic 1 ------
 police oakland #oo protester park cops city news nypd

------ Topic 2 ------
 right movement #n17 world one freedom make protest change

------ Topic 3 ------
 street occupy wall nyc temp forecast high la night

------ Topic 4 ------
 tax bank #ronpaul year money pay jobs american class



## Iteration 4 (12 Topics no top words)

In [ ]:
model_iter4 = LdaMulticore(corpus=corpus, num_topics=12, id2word=id2word, passes=5, random_state = 100)


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter4, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter4 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter4)


Coherence Score:  0.3855932373700904


In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_iter4.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
park zuccotti bloomberg one mayor news nypd protester court back

------ Topic 1 ------
#ronpaul money tax bank pay #nycga class american economy million

------ Topic 2 ------
1% 99% power corporate truth capitalism world american time government

------ Topic 3 ------
tweet last twitter news check medium night new live see

------ Topic 4 ------
occupy street wall party obama st tea union movement bank

------ Topic 5 ------
police protester cops arrest arrested oakland nypd tent peaceful protest

------ Topic 6 ------
#oo oakland la forecast #n17 #opdx sf #osf revolution de

------ Topic 7 ------
#solidarity us day today #nycsc stay toher tonight action #oo

------ Topic 8 ------
#teaparty nyc temp #tlot #gop high low #ronpaul night #topprog

------ Topic 9 ------
big world never change want go system paul god let

------ Topic 10 ------
right #ronpaul #ndaa #teaparty freedom vote #anonymous obama #sopa #gop

------ Topic 11 ------
think know thing want work wo

In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter4, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.056474  0.129342       1        1  16.529681
7      0.133943  0.058641       2        1  10.153500
10    -0.181787  0.013778       3        1  10.011130
1     -0.184773  0.004499       4        1   9.573923
5      0.164010  0.086651       5        1   9.446500
3      0.149273  0.091303       6        1   8.147919
2     -0.183469  0.005139       7        1   7.896091
0      0.151663  0.117850       8        1   6.545662
4      0.035948  0.049266       9        1   6.245503
9     -0.069525  0.042375      10        1   5.709290
8     -0.155883 -0.218740      11        1   4.980901
6      0.197074 -0.380103      12        1   4.759900, topic_info=            Term          Freq         Total Category  logprob  loglift
66906        #oo  47971.000000  47971.000000  Default  30.0000  30.0000
172       police  53748.000000  53748.000000  Default  29.0000  29.0000
466       occupy  45866.000000  45866.000000  Default  28.0000  28.0000
94        street  36066.000000  36066.000000  Default  27.0000  27.0000
62     #teaparty  49011.000000  49011.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
166         make   2286.771349  34718.890413  Topic12  -5.5061   0.3248
1087         ows   1883.178236   8193.690093  Topic12  -5.7002   1.5745
227          big   2031.653074  18417.833090  Topic12  -5.6244   0.8405
2406           1   1796.301113   6545.110523  Topic12  -5.7475   1.7520
465          new   1868.248936  27213.059300  Topic12  -5.7082   0.3662

[884 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
3768      12  0.999421      #15m
11507      9  0.000732     #2012
11507     10  0.000732     #2012
11507     11  0.997858     #2012
1492       1  0.014629      #99%
...      ...       ...       ...
2593       6  0.998945   zionist
665        5  0.018114  zuccotti
665        6  0.000107  zuccotti
665        8  0.981584  zuccotti
3314       8  0.999704   zucotti

[3371 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 8, 11, 2, 6, 4, 3, 1, 5, 10, 9, 7])

## Iteration 5 (9 topics no top words)

In [ ]:
model_iter5 = LdaMulticore(corpus=corpus, num_topics=9, id2word=id2word, passes=5, random_state = 100)


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter5, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter5 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter5)


Coherence Score:  0.3287114710371063


In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_iter5.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
one know #ndaa think #opdx #sopa time want right would

------ Topic 1 ------
right need state class make change #nycga #ronpaul revolution must

------ Topic 2 ------
#ronpaul 1% tax government 99% american #teaparty money want problem

------ Topic 3 ------
news last tweet twitter night live tonight today medium ga

------ Topic 4 ------
street obama wall occupy party movement st would tea support

------ Topic 5 ------
police protester park cops nypd arrest city arrested protest violence

------ Topic 6 ------
bank big la money union business de student credit #99%

------ Topic 7 ------
#oo oakland #n17 us #solidarity day love world stay back

------ Topic 8 ------
#teaparty #ronpaul nyc #gop temp #tlot forecast high low night



In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter5, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.060404  0.030613       1        1  13.486925
2      0.145521  0.018551       2        1  13.399505
5     -0.183099 -0.009508       3        1  12.446634
7     -0.139382 -0.000759       4        1  12.114950
3     -0.205373  0.004400       5        1  11.639132
4     -0.039865  0.054279       6        1  11.511335
1      0.063890  0.008820       7        1  11.447606
8      0.247727  0.234546       8        1   7.337488
6      0.170987 -0.340942       9        1   6.616426, topic_info=            Term          Freq         Total Category  logprob  loglift
172       police  53993.000000  53993.000000  Default  30.0000  30.0000
66906        #oo  47600.000000  47600.000000  Default  29.0000  29.0000
62     #teaparty  47718.000000  47718.000000  Default  28.0000  28.0000
94        street  35373.000000  35373.000000  Default  27.0000  27.0000
2381    #ronpaul  55282.000000  55282.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
3537       local   2556.776592   6136.032675   Topic9  -5.7238   1.8402
1203        free   2872.549965  18789.214724   Topic9  -5.6073   0.8375
167         want   2954.547204  38836.681199   Topic9  -5.5792   0.1396
465          new   2663.191881  27066.685385   Topic9  -5.6830   0.3968
88          bill   2563.101745   7964.390127   Topic9  -5.7213   1.5819

[685 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3768      9  0.999519      #15m
1492      2  0.062951      #99%
1492      6  0.002308      #99%
1492      7  0.000210      #99%
1492      9  0.934190      #99%
...     ...       ...       ...
1550      1  0.998638      zone
1550      5  0.000916      zone
665       1  0.000775  zuccotti
665       3  0.982847  zuccotti
665       5  0.016274  zuccotti

[2341 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 8, 4, 5, 2, 9, 7])

## Iteration 6 (7 topics no top words)

In [ ]:
model_iter6 = LdaMulticore(corpus=corpus, num_topics=7, id2word=id2word, passes=5, random_state = 100)


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter6, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter6 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter6)


Coherence Score:  0.30738734226027004


In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_iter6.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
think one know would go want time really job #opdx

------ Topic 1 ------
money tax #ndaa need #ronpaul bank congress make pay corporate

------ Topic 2 ------
#ronpaul #teaparty #anonymous #gop government us 99% 1% american need

------ Topic 3 ------
#oo oakland #nycga today thanks tweet occupy #nycsc twitter la

------ Topic 4 ------
street wall nyc occupy temp party high low obama night

------ Topic 5 ------
police protester park cops nypd oakland arrest city arrested right

------ Topic 6 ------
forecast #n17 revolution la #ronpaul world de civil right make



In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter6, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.060275  0.017692       1        1  19.412714
1      0.106359 -0.042565       2        1  16.831842
5     -0.147700  0.150466       3        1  16.801837
3     -0.186884  0.125456       4        1  15.596750
2      0.155393 -0.040439       5        1  14.444439
4     -0.133785 -0.283914       6        1   9.022907
6      0.266891  0.073303       7        1   7.889511, topic_info=             Term          Freq         Total Category  logprob  loglift
172        police  54149.000000  54149.000000  Default  30.0000  30.0000
66906         #oo  46888.000000  46888.000000  Default  29.0000  29.0000
94         street  32254.000000  32254.000000  Default  28.0000  28.0000
2381     #ronpaul  64026.000000  64026.000000  Default  27.0000  27.0000
95           wall  26080.000000  26080.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
121    #anonymous   3035.593122  29526.156508   Topic7  -5.7281   0.2648
1926         must   2784.436737  17262.167575   Topic7  -5.8145   0.7152
411           99%   2802.977420  21228.039858   Topic7  -5.8078   0.5150
465           new   2760.695644  26645.843231   Topic7  -5.8230   0.2725
302          work   2662.287792  23502.006309   Topic7  -5.8593   0.3617

[574 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3768      7  0.999132      #15m
1492      1  0.000233      #99%
1492      2  0.002095      #99%
1492      3  0.000466      #99%
1492      4  0.000233      #99%
...     ...       ...       ...
665       1  0.013942  zuccotti
665       3  0.984283  zuccotti
665       4  0.001785  zuccotti
3314      1  0.999008   zucotti
3314      4  0.000676   zucotti

[1728 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 6, 4, 3, 5, 7])

## Iteration 7 (8 topics no top words)

In [ ]:
model_iter7 = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, passes=5, random_state = 100)


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter7, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter7 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter7)


Coherence Score:  0.3138890249469024


In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_iter7.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
#ronpaul #opdx medium #teaparty one time big news us said

------ Topic 1 ------
#ronpaul #ndaa government #teaparty american state war #anonymous usa money

------ Topic 2 ------
nyc temp forecast #teaparty #gop #tlot tax high low night

------ Topic 3 ------
oakland #nycga occupy today #nycsc new last twitter tonight night

------ Topic 4 ------
obama street wall occupy movement think party would know support

------ Topic 5 ------
police protester park cops nypd arrest arrested right violence protest

------ Topic 6 ------
want #n17 1% free 99% bank world money work make

------ Topic 7 ------
#oo love us go #solidarity stay day thanks #sopa back



In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter7, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.055360  0.079625       1        1  17.228733
6      0.006473  0.161279       2        1  15.042704
3     -0.158936 -0.185629       3        1  14.222628
5     -0.152277 -0.099259       4        1  13.846020
1      0.104256  0.176998       5        1  12.509079
7     -0.117727 -0.062940       6        1  11.910583
0     -0.010033  0.102217       7        1   8.495067
2      0.383603 -0.172291       8        1   6.745186, topic_info=           Term          Freq         Total Category  logprob  loglift
172      police  54349.000000  54349.000000  Default  30.0000  30.0000
66906       #oo  46856.000000  46856.000000  Default  29.0000  29.0000
2255    oakland  37367.000000  37367.000000  Default  28.0000  28.0000
2381   #ronpaul  57641.000000  57641.000000  Default  27.0000  27.0000
437         nyc  31053.000000  31053.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
604          la   4904.360074  24573.229081   Topic8  -5.0917   1.0848
2381   #ronpaul   6293.989140  57641.943958   Topic8  -4.8422   0.4817
1500       #usa   2804.344546   5088.432844   Topic8  -5.6506   2.1005
2389         1%   4097.409309  25627.647314   Topic8  -5.2715   0.8630
3862     #obama   2759.387375   9148.098281   Topic8  -5.6668   1.4978

[613 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1492      1  0.021370      #99%
1492      2  0.221658      #99%
1492      3  0.002728      #99%
1492      4  0.001364      #99%
1492      5  0.008866      #99%
...     ...       ...       ...
665       4  0.967343  zuccotti
665       7  0.000995  zuccotti
3314      3  0.012553   zucotti
3314      4  0.984526   zucotti
3314      7  0.002714   zucotti

[1924 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 4, 6, 2, 8, 1, 3])

## Iteration 8 (8 topics no top words passes = 20)

In [ ]:
model_iter8 = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, passes=20, random_state = 100)


In [ ]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=model_iter8, texts=model_ows['lemmatized_tokens'], dictionary=id2word, coherence='c_v')
coherence_lda_model_iter8 = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_iter8)


Coherence Score:  0.36041482169241956


In [ ]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model_iter8.print_topics()]

In [ ]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [ ]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
#ronpaul #teaparty #anonymous freedom free #ndaa government #opdx liberty right

------ Topic 1 ------
#ronpaul tax usa war state american congress #teaparty class pay

------ Topic 2 ------
nyc temp forecast #gop #tlot #teaparty high low night #topprog

------ Topic 3 ------
oakland occupy news #nycga today #nycsc twitter last la action

------ Topic 4 ------
obama street wall think occupy movement party would know right

------ Topic 5 ------
police protester park cops nypd arrest right arrested violence protest

------ Topic 6 ------
want 1% 99% #n17 work make money need one bank

------ Topic 7 ------
#oo love world us go good keep back stay #solidarity



------ Topic 0 (Other Political Groups and Movements)------ 

\#ronpaul #teaparty #anonymous freedom free #ndaa government #opdx liberty right

------ Topic 1 (Policy Discussions)------

\#ronpaul tax usa war state american congress #teaparty class pay

------ Topic 2 (Daily Updates)------

nyc temp forecast #gop #tlot #teaparty high low night #topprog

------ Topic 3 (Occupying Other Spaces)------
\
oakland occupy news #nycga today #nycsc twitter last la action

------ Topic 4 (Noisy Topic)------

obama street wall think occupy movement party would know right

------ Topic 5 (Police Violence)------

police protester park cops nypd arrest right arrested violence protest

------ Topic 6 (Occupy Strategy and Theory)------

want 1% 99% #n17 work make money need one bank

------ Topic 7 (Solidarity and Support)------

\#oo love world us go good keep back stay #solidarity


In [ ]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_iter8, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.131435 -0.022685       1        1  17.836159
6      0.135600  0.093350       2        1  16.095423
7      0.126611 -0.084561       3        1  14.098502
3      0.072394 -0.220986       4        1  13.959373
5      0.089405 -0.142420       5        1  13.513706
1     -0.016159  0.231600       6        1  11.295140
0     -0.084853  0.269807       7        1   7.253539
2     -0.454433 -0.124104       8        1   5.948157, topic_info=            Term          Freq         Total Category  logprob  loglift
2381    #ronpaul  59907.000000  59907.000000  Default  30.0000  30.0000
172       police  54104.000000  54104.000000  Default  29.0000  29.0000
66906        #oo  47186.000000  47186.000000  Default  28.0000  28.0000
62     #teaparty  55073.000000  55073.000000  Default  27.0000  27.0000
2255     oakland  37623.000000  37623.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1500        #usa   3817.787159   5391.089598   Topic8  -5.2164   2.4770
62     #teaparty  14281.822298  55073.890874   Topic8  -3.8971   1.4724
604           la   5892.697171  24817.382768   Topic8  -4.7823   1.3843
3862      #obama   3909.867073   9060.671562   Topic8  -5.1926   1.9816
2271      friday   2638.298074   5774.661045   Topic8  -5.5859   2.0387

[490 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1492      2  0.177747        #99%
1492      6  0.822240        #99%
1497      8  0.999551    #america
121       3  0.233731  #anonymous
121       6  0.215460  #anonymous
...     ...       ...         ...
234       1  0.048298        year
234       3  0.517530        year
234       6  0.434107        year
665       5  0.999871    zuccotti
3314      5  0.999713     zucotti

[846 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 8, 4, 6, 2, 1, 3])

In [ ]:
model_iter8.save('/content/drive/MyDrive/Colab Notebooks/ows_twitter.model')

# Split Tweets by Topic

In [ ]:
doc_lda = model_iter8[corpus]
topic_scores = [[topic_score[1] for topic_score in sent] for sent in doc_lda]
df_topics = pd.DataFrame(topic_scores)
max_topics = [max(sent, key=lambda x: x[1])[0] for sent in doc_lda]


In [ ]:
model_ows['Topic'] = max_topics

In [6]:
model_ows.sample(20)

,Unnamed: 0,Twitter ID,Text,Day,Created At,clean_text,text_lemmatized,lemmatized_tokens,is_retweet,has_multimedia,Topic
1177288,74987748,162017557287673856,#OccupySalem's Long Range Planning is putting ...,2012-01-25,2012-01-25 03:42:50,#occupysalem long range planning is putting to...,"['#occupysalem', 'long', 'range', 'planning', ...","['salem', 'long', 'range', 'planning', 'puttin...",False,False,3
52605,1055688,123170139708461056,#OccupyMadison has been permitted to stay for ...,2011-10-09,2011-10-09 22:57:04,#occupymadison has been permitted to stay for ...,"['#occupymadison', 'has', 'been', 'permitted',...","['madison', 'permitted', 'stay', 'another', 'd...",False,False,7
301397,14996545,127458160163291136,@ladykayaker I agree! NYPD make a twitter so w...,2011-10-21,2011-10-21 18:56:08,agree nypd make twitter so whatever you nee...,"['agree', 'nypd', 'make', 'twitter', 'so', 'wh...","['agree', 'nypd', 'make', 'twitter', 'whatever...",False,False,6
722768,35556496,137164012952621056,protestors sing &quot;we shall overcome&quot; ...,2011-11-17,2011-11-17 13:43:43,protestors sing we shall overcome at wall stre...,"['protestors', 'sing', 'we', 'shall', 'overcom...","['protestors', 'sing', 'shall', 'overcome', 'w...",False,False,4
813328,42895505,138060750492147712,@MMFlint getting income inequality into nation...,2011-11-20,2011-11-20 01:07:02,getting income inequality into national dial...,"['getting', 'income', 'inequality', 'into', 'n...","['ting', 'income', 'inequality', 'national', '...",False,False,1
182703,11141801,125375616274006016,#occupytcot @TruthTeller1125 @weshouldallcare ...,2011-10-16,2011-10-16 01:00:50,#occupytcot job creators got tax cuts people ...,"['#occupytcot', 'job', 'creator', 'got', 'tax'...","['tcot', 'job', 'creator', 'got', 'tax', 'cuts...",False,False,6
858905,46828003,139086880019976192,#OWS #OccupyOakland CS-Gas or pepper spray is ...,2011-11-22,2011-11-22 21:04:30,#ows #occupyoakland cs gas or pepper spray is ...,"['#ows', '#occupyoakland', 'cs', 'gas', 'or', ...","['oakland', 'cs', 'gas', 'pepper', 'spray', 'h...",False,False,7
985102,61878277,144569262764331008,Per @KFOR and @Occupy_Oklahoma: No decision by...,2011-12-08,2011-12-08 00:09:32,per and no decision by federal court on #occup...,"['per', 'and', 'no', 'decision', 'by', 'federa...","['per', 'decision', 'federal', 'court', 'okc',...",False,False,5
652428,30417759,136452589834346496,CORRECTION 180-190 #occupyarrests according to...,2011-11-15,2011-11-15 14:36:47,correction #occupyarrests according to nyt at...,"['correction', '#occupyarrests', 'according', ...","['correction', 'arrests', 'according', 'nyt', ...",False,False,3
736237,36220575,137208144295239680,soooooo.... 320 million people in America are ...,2011-11-17,2011-11-17 16:39:05,soooooo million people in america are still pr...,"['soooooo', 'million', 'people', 'in', 'americ...","['soooooo', 'million', 'america', 'still', 'pr...",False,False,7


In [ ]:
model_ows.to_csv('/content/drive/MyDrive/Colab Notebooks/ows_tweet_topics.csv')